In [786]:
from pyscopus import Scopus
import pandas as pd
import requests
import xmltodict, json


from lxml import etree
import requests
import xmltodict, json

from bs4 import BeautifulSoup
import numpy as np

In [787]:
print("Ingrese el dato a buscar:")
busqueda = input()

Ingrese el dato a buscar:
medicine


In [788]:
#Consumo API de SCOPUS

In [789]:
key = '0387c7142db3254ed53c09c131ecf8c0'
scopus = Scopus(key)
search_df = scopus.search("ALL("+busqueda+")", count=100)

In [790]:
#Consumo API de DOAJ

In [791]:
#pasar a ingles 
#funciona con el spacio 
url = 'https://www.doaj.org/api/v2/search/articles/all('+busqueda+')?pageSize=100'
headers = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r = requests.get(url, headers=headers)

In [792]:
#Consumo API de propiedad Inletectual

In [793]:
#api lenta 
#funciona con spacio 
#ingle
url_uspto = 'https://developer.uspto.gov/ibd-api/v1/application/publications?searchText='+busqueda+'&start=0&rows=100'
headers_uspto = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r2_uspto = requests.get(url_uspto, headers=headers_uspto) 

In [794]:
#Consumo API deProQuest

In [795]:
access_token = '8461684a-5a79-407c-9321-93d8d719ddeb'
headers = {
    'Content-type': 'text/xml',
    'Authorization': 'Bearer {}'.format(access_token)
}
data = '<searchRequest><search><query>'+busqueda+'</query><databases><database>medlineprof</database></databases></search><count>100</count></searchRequest>'
resp = requests.post('https://api-dialog.proquest.com/v1/search', headers=headers, data=data)
o = xmltodict.parse(resp.text)
conta = json.dumps(o)
d = json.loads(conta)


In [796]:
#Consumo API de IEEEXPLORE

In [797]:
#funciona con spacio 
#debe estar en ingles 

url_ieeexplore = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=3dyyvg36jv78dbuu83q3fzzt&format=json&max_records=200&start_record=1&sort_order=asc&sort_field=article_number&querytext='+busqueda
headers_ieeexplore = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r_ieeexplore = requests.get(url_ieeexplore, headers=headers_ieeexplore)

In [798]:
#Consumo Google Noticas

In [799]:
busqueda =busqueda.replace(" ", "+")
res = requests.get('https://www.google.com/search?q='+busqueda+'+&biw=1064&bih=1034&tbm=nws&sxsrf=ALeKk012okzl46BThz3OHzeulw_CfNeALg%3A1620065628921&ei=XD2QYN2hN52y5NoP88anwAw&oq='+busqueda+'+&gs_l=psy-ab.3...1712.7789.0.7910.36.29.2.0.0.0.336.3435.0j13j5j1.20.0....0...1c.1.64.psy-ab..15.11.1630.0..0.124.YgObgGkRCac')
soup = BeautifulSoup(res.text, 'lxml')

In [800]:
#Declaracion 
listalinks = list()
listatitulos = list()
listabstract = list()
listautor= list()
listakeywords = list()
listafuente = list()


In [801]:
#Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
content_ieeexplore = r_ieeexplore.json()
json_ieeexplore = content_ieeexplore['articles']
for i in range(len(json_ieeexplore)):
    try:
        listabstract.append(json_ieeexplore[i]['abstract'])
        listatitulos.append(json_ieeexplore[i]['title'])
        listalinks.append(json_ieeexplore[i]['pdf_url'])
        aux=list()
        for j in range(len(json_ieeexplore[i]['authors']['authors'])):
            aux.append(json_ieeexplore[i]['authors']['authors'][j]['full_name'])
        listautor.append(aux)
        try:
            listakeywords.append(json_ieeexplore[i]['index_terms']['ieee_terms']['terms'])
        except:
            listakeywords.append('0')    
        listafuente.append('ieeexplore')   
    except:
        print("Error sin abstract")

Error sin abstract
Error sin abstract


In [802]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 198  title  198 abstract  198  authors  198  keywords  198 fuente 198


In [803]:
#Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
content_uspto = r2_uspto.json()
json_uspto = content_uspto['results']

for i in range(len(json_uspto)):
    try:

        listakeywords.append(0)
        listafuente.append("uspto")
        listatitulos.append(json_uspto[i]['inventionTitle'])
        listalinks.append(json_uspto[i]['filelocationURI'])
        listabstract.append(nivel3)
        listautor.append(json_uspto[i]['abstractText'][0])
    except:
        listautor.append('NULL')
        listabstract.append('NULL')


In [804]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 298  title  298 abstract  298  authors  298  keywords  298 fuente 298


In [805]:
#Consumo api  de doaj sobre titulo,url,abstract,author(uspto),keywords
content = r.json()
#Ingreso al contenido
json = content['results']

for i in range(len(json)):
    try:
        listakeywords.append(json[i]['bibjson']['keywords'])
        listalinks.append(json[i]['bibjson']['link'][0]['url'])
        listatitulos.append(json[i]['bibjson']['title'])
        listabstract.append(json[i]['bibjson']['abstract'])
        listautor.append(json[i]['bibjson']['author'])
        listafuente.append("doaj")
    except:
        print("")

In [806]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 370  title  370 abstract  370  authors  370  keywords  370 fuente 370


In [807]:
#Consumo api  de SCOPUS sobre titulo,url,abstract,nombre(uspto)
for i in range(len(search_df['scopus_id'])):
    try:
        pub_abs = scopus.retrieve_abstract(search_df['scopus_id'][i])
        listabstract.append(pub_abs['abstract'])
        listalinks.append("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp="+search_df['scopus_id'][i]+"&origin=inward")
        listatitulos.append(search_df['title'][i])
        listautor.append(search_df['authors'][i])
        listakeywords.append("0")
        listafuente.append("scopus")
    except:
        print(" ")

In [808]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 445  title  445 abstract  445  authors  445  keywords  445 fuente 445


In [809]:
#Consumo api  de ProQuest sobre titulo,url,abstract,nombre(uspto)
json = d['searchResponse']['result']['doc']

for i in range(len(json)):
        #TITULO
        listatitulos.append(json[i]['field'][1]['value'])
        response2 = requests.get(json[i]['field'][3]['value'], headers=headers)
        o1 = xmltodict.parse(response2.text)
        
        #LINK
        listalinks.append(o1['Documents']['Literature']['DocInfo']['URL'])
        
        #ABSTRACT
        try:
            nivel5= o1['Documents']['Literature']['Abstract']
            listabstract.append(nivel5.get('#text'))
        except:
            listabstract.append("NULL")
        
        #Fuente
        listafuente.append('ProQuest')
        
        
        #Authores
        try:
            contributor=o1['Documents']['Literature']['Contributors']['Contributor']
            aux=list()
            for i in range(len(contributor)):
                if(contributor[i]['@ContribRole']=='Author'):
                    aux.append(contributor[i]['NormalizedName']+" "+contributor[i]['LastName'])
            listautor.append(aux)
        except:
            listautor.append("NULL")

        try:
            #Keywords
            keywords=o1['Documents']['Literature']['Subjects']['HeadingTerms']['HeadingTerm']
            aux=list()
            for i in range(len(keywords)):
                nivel9=keywords[i]['Heading']
                aux.append(nivel9['#text'])
            listakeywords.append(aux)
        except:
            print("error key")
            listakeywords.append('0')
        

In [810]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 545  title  545 abstract  545  authors  545  keywords  545 fuente 545


In [811]:

title_noticias = soup.find_all('div', class_="BNeawe vvjwJb AP7Wnd")
for i in range(len(title_noticias)):
    print(title_noticias[i].text)
    listatitulos.append(title_noticias[i].text)

 
    
aux=list()
abstract_noticias = soup.find_all('div', class_="BNeawe s3v9rd AP7Wnd")
for i in range(len(abstract_noticias)):
    abstract_text=abstract_noticias[i].text
    abstract1=abstract_text[abstract_text.find(' · ')+3:len(abstract_text)]
    limpieza=abstract1.replace('... ','')
    limpieza=limpieza.replace('...','')
    aux.append(limpieza)
    
aux=pd.unique(aux)
listabstract = np.append(listabstract,aux)

aux=list()
link_noticas = soup.find_all('div', class_="kCrYT")

for link_noticas in link_noticas:
    if(str(link_noticas)[35:str(link_noticas).find('&amp')][0:5]=='https'):
        aux.append(str(link_noticas)[35:str(link_noticas).find('&amp')])

aux=pd.unique(aux)
listalinks = np.append(listalinks,aux)


author_noticas = soup.find_all('div', class_="BNeawe UPmit AP7Wnd")
for i in range(10):
    print(author_noticas[i].text)
    listautor.append(author_noticas[i].text)
    listafuente.append('google noticias')
    listakeywords.append('0')



2 Minute Medicine Rewind May 3, 2020
The Future of Medicine Requires Investors Who Take the Long View
National Academy of Medicine Names 10 Inaugural Scholars in ...
BU School of Medicine Adds St. Elizabeth's Medical Center as New ...
Cannabis is medicine. Just ask the parents of children with seizures
Coeur d'Alene (WA) Wilderness Medicine Course Emphasizes Self ...
Personalized Medicine For The Planet
IQVIA study: World medicine spending to hit $1.6 trillion in 2025
Michigan Medicine opens COVID-19 long-haul clinics
Belmont to name College of Medicine in honor of Dr. Thomas F. Frist Jr.
2 Minute Medicine
Barron's
National Academies
Boston University
Toronto Star
JEMS
Forbes
Medical Economics
MLive.com
wsmv.com


In [812]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 555  title  555 abstract  555  authors  555  keywords  555 fuente 555


In [813]:
diccionario ={'link':listalinks ,'title':listatitulos,'abstract':listabstract,'authors':listautor,"keywords":listakeywords,'fuente':listafuente}

In [814]:
dataset=pd.DataFrame(diccionario)
print(dataset)

                                                  link  \
0    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
1    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
2    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
3    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
4    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
..                                                 ...   
550  https://www.jems.com/news/coeur-dalene-wa-wild...   
551  https://www.forbes.com/sites/jeffopperman/2021...   
552  https://www.medicaleconomics.com/view/iqvia-st...   
553  https://www.mlive.com/news/ann-arbor/2021/05/m...   
554  https://www.wsmv.com/news/davidson_county/belm...   

                                                 title  \
0    Simulations of foot stability during gait char...   
1     Reciprocal EMG control of elbow extension by FES   
2    Finite element modeling of electromagnetic sig...   
3    Interaction of array of finite electrodes with...   
4    Kinemati

In [815]:
dataset.to_csv('resultado.csv')

In [466]:
clear